In [93]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
from urllib.parse import urljoin

In [94]:
# Get Becker's Hospital Review URL
HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
url = "https://www.beckershospitalreview.com/healthcare-information-technology"
newsurl = "https://www.beckershospitalreview.com"
page = requests.get(url, headers=HEADERS)

# Get request status
status = page.status_code
if status == 403:
    raise ValueError("Exception thrown as url get request returned 403 error. No access")

# Open list of keywords
with open('keywords.json') as file:
     data = json.load(file)

In [95]:
# Parse html code
# We'll save in coverpage the cover page content
beckerHospitalIT = page.content

# Soup creation
soup1 = BeautifulSoup(beckerHospitalIT, 'html5lib') # using html parser to read through site data

In [96]:
# Scrape html from site
## Source 3 - Hippa Journal ##
# News identification for Source 1
coverpage_news = soup1.find_all('div', id='content')       # Select right column with latest news
if not coverpage_news:
    # Throws exception if soup cannot find the main 'container' on the site
    # Only returns exception if the length of coverpage news is 0 ... should be 1 and not 0
    raise ValueError("Error in returning html data from main page. Length of array is 0")

# Accesses direct list of srticles from the page wrapper html id
# Using only one find all instead of several unnecessary calls    
articles = coverpage_news[0].contents[1].contents[11].contents[1].contents[3].find_all('li')

In [97]:
# Establish the number of articles
num_of_articles = len(articles)
if not num_of_articles:
    # Throws exception if soup cannot find articles in find_all
    # Only returns exception if the length of articles is 0 ... should be more than 0
    raise ValueError("Error in returning html data. Length of array is 0")

In [98]:
Index = []
list_links = []
list_titles = []
Keyword = []
count = 0
added_articles = set()

# for each article in list of scraped articles 
for n in np.arange(0, num_of_articles):
    
    # Getting the link of the article
    link = articles[n].find('a')['href']
    absolute_url = urljoin(newsurl, link)
    
    
    # Getting the title
    title = articles[n].get_text()
    title = title.replace('\n', '') # Remove \n
    title = title.replace('\t', '') # Remove \t
    title = " ".join(title.split()) # Remove spaces in front and back of string
    
    found_keyword = False # initialize found_keyword flag
    
# https://www.geeksforgeeks.org/read-json-file-using-python/
# Filter by Keyword inside "data"
    for entry in data:
        if entry['Keyword'] in title:
            Keyword.append(entry['Keyword'])  # Append Found Keyword
            added_articles.add(title)
            found_keyword = True # set found_keyword flag to True
            break
    
    # append 'none' if no keyword was found
    if not found_keyword:
        Keyword.append('none')
    
    # add article information to lists
    list_links.append(absolute_url)
    list_titles.append(title)
    Index.append(count)
    count += 1 

In [99]:
 # df_show_info
df_show_info = pd.DataFrame(
    {'ind': Index,
     'keyword': Keyword,
     'Article Title': list_titles,
     'Article Link': list_links})

df_show_info = df_show_info.drop_duplicates()
df_show_info

,ind,keyword,Article Title,Article Link
0,0,none,5 hospitals and health systems choosing Epic EHRs,https://www.beckershospitalreview.com/ehrs/5-h...
1,1,none,'Suspicious communication' leads Norton to shu...,https://www.beckershospitalreview.com/cybersec...
2,2,none,Can generative AI revolutionize the EHR? ChatG...,https://www.beckershospitalreview.com/ehrs/can...
3,3,none,The 'non-negotiable' areas for health system I...,https://www.beckershospitalreview.com/healthca...
4,4,none,VA pharmacists say Cerner EHR system increases...,https://www.beckershospitalreview.com/ehrs/va-...
5,5,none,Which top health systems are still verified on...,https://www.beckershospitalreview.com/digital-...
6,6,cyberattacks,5 health systems experiencing cyberattacks,https://www.beckershospitalreview.com/cybersec...
7,7,none,Where Cerner is partnering in 2023,https://www.beckershospitalreview.com/ehrs/whe...
8,8,none,Oregon VA system to reset Cerner EHR,https://www.beckershospitalreview.com/ehrs/ore...
9,9,none,Vets call for end to Cerner rollout at Washing...,https://www.beckershospitalreview.com/ehrs/vet...


In [100]:
df_show_info['Article Link']

0    https://www.beckershospitalreview.com/ehrs/5-h...
1    https://www.beckershospitalreview.com/cybersec...
2    https://www.beckershospitalreview.com/ehrs/can...
3    https://www.beckershospitalreview.com/healthca...
4    https://www.beckershospitalreview.com/ehrs/va-...
5    https://www.beckershospitalreview.com/digital-...
6    https://www.beckershospitalreview.com/cybersec...
7    https://www.beckershospitalreview.com/ehrs/whe...
8    https://www.beckershospitalreview.com/ehrs/ore...
9    https://www.beckershospitalreview.com/ehrs/vet...
Name: Article Link, dtype: object

# MySQL Integration

## Make sure that pymysql is installed!!

This next section will be placing news article Titles and URLs into our databse which will then be displayed on the site using queries.

In [101]:
import pymysql

# Connect to the database with credentials
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='password',
                         db='medcurity-db')

# create cursor
cursor=connection.cursor()

# initialize the data we are pushing into the DB as the table created above
data = df_show_info

RuntimeError: 'cryptography' package is required for sha256_password or caching_sha2_password auth methods

In [ ]:
# add rows/tuples into table (append procees no overwrite)
# df_show_info.columns = ['ind','keyword', 'articleTitle','articleLink']
cols = "`,`".join([str(i) for i in data.columns.tolist()])
for i,row in data.iterrows():
    
    # Extract the relevant data from each row
    title = row['Article Title'].replace("'", "''")
    url = row['Article Link']
    
    # Construct the SQL query string with placeholders for the variables
    sql = "CALL insert_news_article('{}', '{}');".format(title, url)
    
    # Execute the SQL query string using the database cursor
    cursor.execute(sql)
    
    # Commit the changes to the database
    connection.commit()